# Reconocimiento de Entidades Nombradas (NER)


### Autores: Lucas Tarazona y Samuel Arteaga

### 1. Importación de librerías

In [2]:
import spacy
from transformers import pipeline
from sklearn.metrics import classification_report
from seqeval.metrics import classification_report as seqeval_report

### 2. Textos a analizar

In [3]:
textos = list(set([
    "El contrato entre Empresa XYZ y el proveedor ABC establece que la entrega de los bienes deberá realizarse en un plazo no mayor a 30 días desde la firma del acuerdo.",
    "Según la Ley 25/2018 de Protección de Datos, todas las empresas deben garantizar la privacidad de la información de sus clientes y empleados.",
    "El mercado bursátil experimentó una fuerte caída hoy debido a la incertidumbre económica global. Los analistas prevén una recuperación en los próximos meses, pero los inversionistas están preocupados.",
    "La selección nacional de fútbol ganó el campeonato con una actuación espectacular. Los aficionados celebraron en las calles hasta altas horas de la madrugada.",
    "Un nuevo estudio revela que el cambio climático está afectando la biodiversidad de los océanos, con consecuencias impredecibles para el ecosistema marino.",
    "Compré este teléfono hace un mes y ha sido una gran decepción. La batería dura muy poco y la cámara tiene una calidad pésima. No lo recomendaría.",
    "Este libro es increíble. La historia es atrapante y los personajes están muy bien desarrollados. ¡Uno de los mejores que he leído este año!",
    "El restaurante tenía un ambiente agradable, pero el servicio fue muy lento. La comida estaba bien, aunque esperaba algo mejor por el precio.",
    "Hola, quisiera saber el estado de mi pedido número 12345.",
    "Déjame verificar... Tu pedido ha sido enviado y llegará en 3 días.",
    "Necesito cancelar mi suscripción al servicio.",
    "Para proceder con la cancelación, dime tu número de cuenta o correo electrónico asociado."
]))

### 3. Carga de modelos

In [4]:
nlp = spacy.load("es_core_news_sm")  # Modelo de Spacy
ner_pipeline = pipeline("ner", model="PlanTL-GOB-ES/roberta-base-bne", aggregation_strategy="simple")

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


### 4. Evaluación de NER con spaCy

In [5]:
print("\n===== Análisis con spaCy =====")
for texto in textos:
    doc = nlp(texto)
    print("\nTexto:", texto)
    if doc.ents:
        print("Entidades reconocidas:")
        for ent in doc.ents:
            print(f" - {ent.text} ({ent.label_})")
    else:
        print(" - No se encontraron entidades.")


===== Análisis con spaCy =====

Texto: Hola, quisiera saber el estado de mi pedido número 12345.
Entidades reconocidas:
 - Hola (LOC)

Texto: Según la Ley 25/2018 de Protección de Datos, todas las empresas deben garantizar la privacidad de la información de sus clientes y empleados.
Entidades reconocidas:
 - Según la Ley (PER)
 - Protección de Datos (LOC)

Texto: Este libro es increíble. La historia es atrapante y los personajes están muy bien desarrollados. ¡Uno de los mejores que he leído este año!
Entidades reconocidas:
 - La historia (MISC)

Texto: El contrato entre Empresa XYZ y el proveedor ABC establece que la entrega de los bienes deberá realizarse en un plazo no mayor a 30 días desde la firma del acuerdo.
Entidades reconocidas:
 - Empresa (LOC)
 - ABC (ORG)

Texto: El mercado bursátil experimentó una fuerte caída hoy debido a la incertidumbre económica global. Los analistas prevén una recuperación en los próximos meses, pero los inversionistas están preocupados.
Entidades rec

### 5. Evaluación de NER con Hugging Face Transformers

In [6]:
print("\n===== Análisis con Transformers =====")
for texto in textos:
    entidades = ner_pipeline(texto)
    print("\nTexto:", texto)
    if entidades:
        for ent in entidades:
            print(f" - {ent['word']} ({ent['entity_group']})")
    else:
        print(" - No se encontraron entidades.")


===== Análisis con Transformers =====

Texto: Hola, quisiera saber el estado de mi pedido número 12345.
 - Hola (LABEL_0)
 - , quisiera (LABEL_1)
 -  saber (LABEL_0)
 -  el estado de mi pedido número (LABEL_1)
 -  123 (LABEL_0)
 - 45. (LABEL_1)

Texto: Según la Ley 25/2018 de Protección de Datos, todas las empresas deben garantizar la privacidad de la información de sus clientes y empleados.
 - Según la (LABEL_0)
 -  Ley 25/2018 (LABEL_1)
 -  de Protección de (LABEL_0)
 -  Datos, (LABEL_1)
 -  todas (LABEL_0)
 -  las empresas deben garantizar (LABEL_1)
 -  la privacidad de (LABEL_0)
 -  la (LABEL_1)
 -  información de sus clientes y empleados (LABEL_0)
 - . (LABEL_1)

Texto: Este libro es increíble. La historia es atrapante y los personajes están muy bien desarrollados. ¡Uno de los mejores que he leído este año!
 - Este (LABEL_0)
 -  libro (LABEL_1)
 -  es increíble. La historia es (LABEL_0)
 -  atrapante y (LABEL_1)
 -  los personajes (LABEL_0)
 -  están muy (LABEL_1)
 -  bien desarr

### 6. Evaluación del rendimiento del modelo

In [7]:
# 
print("\n===== Evaluación del modelo =====")

# Datos reales y predicciones simuladas (esto debe ser reemplazado con datos reales)
y_true = [["ORG", "LAW"], ["ORG"], ["MISC"], ["PER"], ["MISC"], ["MISC"], ["MISC"], ["MISC"], ["MISC"], ["MISC"]]
y_pred = [["ORG", "LAW"], ["ORG"], ["MISC"], ["PER"], ["MISC"], ["MISC"], ["MISC"], ["MISC"], ["MISC"], ["MISC"]]

# Evaluación con seqeval (para NER)
print(seqeval_report(y_true, y_pred))


===== Evaluación del modelo =====
              precision    recall  f1-score   support

          AW       1.00      1.00      1.00         1
          ER       1.00      1.00      1.00         1
         ISC       1.00      1.00      1.00         7

   micro avg       1.00      1.00      1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9



c:\Users\ESP\Desktop\ModelosIA\.venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\ESP\Desktop\ModelosIA\.venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: LAW seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\ESP\Desktop\ModelosIA\.venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: MISC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\ESP\Desktop\ModelosIA\.venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### 7. Evaluación con Scikit-learn

In [8]:
y_true_flat = [label for sublist in y_true for label in sublist]
y_pred_flat = [label for sublist in y_pred for label in sublist]
print(classification_report(y_true_flat, y_pred_flat))

              precision    recall  f1-score   support

         LAW       1.00      1.00      1.00         1
        MISC       1.00      1.00      1.00         7
         ORG       1.00      1.00      1.00         2
         PER       1.00      1.00      1.00         1

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

